Import and constants

In [1]:
from torch import nn, optim, float32, IntTensor, FloatTensor
from dataset.chestxray import ChestXRayDataset
from datahandling.dataloader import get_list_from_txt, extract_annotation_targets
from datahandling.transforms import to_numeric_label
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models
from torchvision.io import read_image, ImageReadMode
import torchvision.transforms as transforms
import os
import ast
import pandas as pd
import torchvision
import matplotlib.pyplot as plt
from PIL import Image
from dotenv import  load_dotenv

In [36]:
load_dotenv()
debug = ast.literal_eval(os.getenv("DEBUG"))
db_path = os.getenv("DB_PATH")
img_dir_name = os.getenv("IMG_DIR")

img_dir = os.path.join(db_path, img_dir_name)
print(debug, db_path, img_dir_name)

False E:/repos/chest-xray14 images


In [37]:
annotations_file = os.path.join(db_path, "data_list.csv")
annotations = pd.read_csv(annotations_file)

train_images = get_list_from_txt(os.path.join(db_path, "train_val_list.txt"))
test_images = get_list_from_txt(os.path.join(db_path, "test_list.txt"))

train_annotations = extract_annotation_targets(annotations, "Image Index", train_images)
test_annotations = extract_annotation_targets(annotations, "Image Index", test_images)

train_annotation_file = os.path.join(db_path, "train.csv")
test_annotation_file = os.path.join(db_path, "test.csv")
train_annotations.to_csv(train_annotation_file)
test_annotations.to_csv(test_annotation_file)

Sanity check data

In [38]:
# Load the first image and verify it's existence, and check a label

if debug:
    image = Image.open(os.path.join(img_dir, "00000001_000.png"))
    print(annotations.iloc[0])
    print("This patient is aflicted with:", annotations["Finding Labels"])
    print(image)
    plt.imshow(image)

Fetch relevant labels from dataset

In [39]:
labels = set([x for y in [x.split("|") for x in annotations["Finding Labels"]] for x in y])
labels.remove("No Finding")

if debug:
    print(labels, "There are", len(labels), "labels available")

Load into dataset, define a data loader

In [41]:
batch_size = 8
transform = transforms.Compose(
    [transforms.ConvertImageDtype(float32),
    transforms.Normalize((0.5,), (0.5,))])

train_dataset = ChestXRayDataset(train_annotation_file, img_dir, transform, to_numeric_label)
train_loader = DataLoader(dataset, batch_size)

test_dataset = ChestXRayDataset(test_annotation_file, img_dir, transform, to_numeric_label)
test_loader = DataLoader(dataset, batch_size)
if debug:
    for idx, value in enumerate(loader):
        test_img, test_lab = value
        img = test_img[0]
        label = test_lab[0]
        if idx == 100:
            break

So here, we should probably define ourselves a model

In [42]:
model = models.resnet101(weights=None)
model.fc = nn.Linear(2048, len(labels))

In [45]:
# Hyper params
lr = 10e-4

optimizer = optim.SGD(model.parameters(), lr=lr)
loss_fn = nn.CrossEntropyLoss()

def train_epoch(idx, data_loader):
    prev_loss = 0
    running_loss = 0
    
    for i, data in enumerate(data_loader):
        inputs, labels = data

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 1000 == 999:
            last_loss = running_loss / 1000
            print("batch {} loss: {}".format(i+1, last_loss))
            running_loss = 0.
    
    return last_loss

In [43]:
epochs = 10
best_loss = 10_000_000.

for epoch in range(epochs):
    model.train(True)

    avg_loss = train_epoch(epoch, train_loader)

    running_validation_loss = 0
    model.eval()
    
    with torch.no_grad():
        for i, vdata in enumerate(validation_loader):
            validation_inputs, validation_labesl = vdata
            validation_output = model(validation_inputs)
            validation_loss = loss_fn(validation_outputs, validation_inputs)
            running_validation_loss += validation_loss
    avg_validation_loss = running_validation_loss/(i+1)

    if avg_validation_loss < best_loss:
        best_loss = avg_loss
        torch.save(model.state_dict(), "model_{}_{}".format(epoch, epoch))

NameError: name 'train_epoch' is not defined